In [ ]:
# fog_node.py

import socket
import pickle
import pandas as pd
import joblib

def load_model_from_file():
    # Load the AI model
    return joblib.load('trained_model.h5')

def load_label_encoders():
    # Load the label encoders used during training
    return joblib.load('label_encoders.pkl')

def preprocess_data(data, label_encoders):
    # Data preprocessing for prediction using the label encoders

    # Convert the data to a DataFrame with a single row
    new_data = pd.DataFrame(data, index=[0])

    # Apply label encoding for categorical attributes based on the loaded encoders
    for column, encoder in label_encoders.items():
        if column in new_data.columns:
            new_data[column] = encoder.transform(new_data[column])

    return new_data

def predict_sleep_disorder(data):
    # Load the AI model
    model = load_model_from_file()

    # Load the label encoders
    label_encoders = load_label_encoders()

    # Data preprocessing for prediction using the label encoders
    new_data = preprocess_data(data, label_encoders)

    # Make predictions using the loaded AI model
    predictions = model.predict(new_data)

    return predictions[0]

def start_fog_node():
    HOST = '172.20.10.6'  # Fog node IP address or '0.0.0.0' to accept connections from any IP
    PORT = 8888     # Fog node port number

    # Load the AI model and label encoders
    model = load_model_from_file()
    label_encoders = load_label_encoders()

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()

        print("Fog node is ready and listening for incoming connections...")

        while True:
            conn, addr = s.accept()
            print("Connected by", addr)

            with conn:
                while True:
                    try:
                        data_bytes = conn.recv(4096)
                        if not data_bytes:
                            break

                        data = pickle.loads(data_bytes)

                        # Process the data using the AI model
                        result = predict_sleep_disorder(data)

                        result_bytes = pickle.dumps(result)
                        conn.sendall(result_bytes)
                    except Exception as e:
                        print("Error:", e)
                        break

            print("response sent")

if __name__ == "__main__":
    start_fog_node()


Fog node is ready and listening for incoming connections...
Connected by ('172.20.10.6', 57830)
response sent
Connected by ('172.20.10.6', 57846)
response sent
Connected by ('172.20.10.6', 57937)
response sent
Connected by ('172.20.10.6', 57946)
response sent
Connected by ('172.20.10.6', 57949)
response sent
Connected by ('172.20.10.6', 57954)
response sent
Connected by ('172.20.10.6', 57957)
response sent
Connected by ('172.20.10.6', 57964)
response sent
